In [1]:
import sys
sys.path.append('../../') # or just install the module
sys.path.append('../../../flaming-choripan') # or just install the module
sys.path.append('../../../astro-lightcurves-handler') # or just install the module

In [2]:
from flamingchoripan.files import search_for_filedirs
from synthsne import C_

root_folder = '../../save'
filedirs = search_for_filedirs(root_folder, fext=C_.EXT_SAMPLER)

────────────────────────────────────────────────────────────────────────────────────────────────────
found filedirs: (../../save)
(0) - ../../save/alerceZTFv7.1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/1@train/samplers.smplr - 10.843[mbs]
(1) - ../../save/alerceZTFv7.1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/2@train/samplers.smplr - 10.715[mbs]
(2) - ../../save/alerceZTFv7.1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/0@train/samplers.smplr - 10.591[mbs]
(3) - ../../save/borra1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/1@train/samplers.smplr - 10.843[mbs]
(4) - ../../save/borra1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/1@val/samplers.smplr - 10.843[mbs]
(5) - ../../save/borra1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/2@train/samplers.smplr - 10.715[mbs]
(6) - ../../save/borra1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/2@val/samplers.smplr - 10.715[mbs]
(7) - ../../save/borra1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/0@train/samplers.smplr - 10.591[mbs]
(8) - ../../save/borr

In [3]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

lcset_name = 'train'
rootdir = f'../../save/alerceZTFv7.1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe/{lcset_name}'
method = 'spm-mle-fstw'

In [8]:
%load_ext autoreload
%autoreload 2
from synthsne.generators.mcmc_priors import GammaP
from synthsne.plots.mcmc import plot_mcmc_prior

gp = GammaP([0,5,6,7,8,9,5,2,78,5,2,2,4,5,0,5,0,1,1])
print(gp.pdf(5))
print(gp.sample(10))
gp

plot_mcmc_prior(gp, 'g')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.06326801921120358
[65.59028139 20.36166378  7.9334465  15.64514309  3.96821896  2.56061818
 17.85957654 24.88401461 59.10879941  7.82649441]


ValueError: 
\gammadist{0.74, 0.08, 1.00}
^
Unknown symbol: \gammadist, found '\'  (at char 0), (line:1, col:1)

ValueError: 
\gammadist{0.74, 0.08, 1.00}
^
Unknown symbol: \gammadist, found '\'  (at char 0), (line:1, col:1)

<Figure size 880x320 with 1 Axes>

In [ ]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms
from flamingchoripan.cuteplots.plots import plot_hist_bins
import matplotlib.pyplot as plt
import scipy
import numpy as np
import scipy.stats

def repr_dist(spm_p, dist_name, dist_params, c, b):
    if dist_name=='norm':
        mu = dist_params[0]
        scale = dist_params[1]
        #txt = f"{spm_p} = pm.Normal('{spm_p}', mu={dist_params[0]}, sigma={dist_params[1]})"
        txt = f"{spm_p} = stats.norm({mu}, {scale})"
        txt += ' # &$\\normdist{'+f'{mu:.2f}'+'}{'+f'{scale:.2f}'+'}$'
    if dist_name=='gamma':
        alpha = dist_params[0]
        mu = dist_params[1]
        scale = dist_params[2]
        beta = 1/scale
        #txt = f"{spm_p} = pm.Gamma('{spm_p}', alpha={dist_params[0]}, beta={beta})+{mu}"
        txt = f"{spm_p} = stats.gamma({alpha}, {mu}, {scale})"
        txt += ' # &$\\gammadist{'+f'{dist_params[0]:.2f}, {beta:.2f}, {mu:.2f}'+'}$'
    return txt

spm_d = {
    'A':('gamma', 0),
    't0':('norm', None),
    'gamma':('gamma', 1),
    #'f':('beta', 0),
    'trise':('gamma', 1),
    'tfall':('gamma', 1),
}
classes = sms.get_classes(rootdir, method)
classes = ['SNIa', 'allSNII', 'SNIbc', 'SLSN']
plot = 1
for c in classes:
    print(f"if c=='{c}':")
    for b in ['g', 'r']:
        print(f"\tif b=='{b}':")
        for spm_p in spm_d.keys():
            spm_args_b = sms.get_spm_args(rootdir, method, b)[spm_p]
            p = np.array([spm_arg['p'] for spm_arg in spm_args_b if c==spm_arg['c']])
            data_dict = {c:p}
            
            dist_name = spm_d[spm_p][0]
            floc = spm_d[spm_p][1]
            if not dist_name is None:
                dist = getattr(scipy.stats, dist_name)
                if dist_name=='gamma':
                    #p = np.clip(p, floc+1e-10, None)
                    #p = p[(p>floc*1.05) & p>floc*1.05)]s
                    p = p[p>floc*1.1]
                fscale = None if dist_name in ['norm', 'gamma'] else 1
                dist_params = dist.fit(p, floc=floc, fscale=fscale)
                x = np.linspace(np.min(p), np.max(p), 100)
                #print(dist_params)
                print(f'\t\t{repr_dist(spm_p, dist_name, dist_params, c, b)}')
                
                if plot:
                    fig, ax = plot_hist_bins(data_dict, uses_density=1, title=spm_p)
                    ax.plot(x, dist.pdf(x, *dist_params), c=b)
                    plt.show()